# pip libarary

In [ ]:
from io import StringIO
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import ast
import time
import copy
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
            

# 算法

In [ ]:
def load_solomon_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if "VEHICLE" in line: 
            vehicle_info_line = lines[i + 2] 
            break

    vehicle_number_line = vehicle_info_line.split()
    vehicle_number = int(vehicle_number_line[0]) 
    vehicle_capacity = int(vehicle_number_line[1]) 
    vehicle_capacitiesLB = [vehicle_capacity] * vehicle_number  

    demandLB = []
    time_windows = []
    coords = []
    service_times = []

    for line in lines[i + 4:]: 
        parts = line.split()
        if len(parts) == 7: 
            customer_id = int(parts[0])
            x_coord = int(parts[1])
            y_coord = int(parts[2])
            demand = int(parts[3])
            ready_time = int(parts[4])
            due_time = int(parts[5])
            service_time = int(parts[6])

            demandLB.append(demand)
            time_windows.append((ready_time, due_time))
            coords.append((x_coord, y_coord))
            service_times.append(int(parts[6]))

    return vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords, service_times

In [ ]:
def calculate_distance_matrix(coords):
    n = len(coords)
    distance_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            dist = math.sqrt((coords[i][0] - coords[j][0]) ** 2 + (coords[i][1] - coords[j][1]) ** 2)
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist 
    return distance_matrix

In [ ]:
def calculate_route_distance(routes, distanceMatrix):
    total_distance = 0
    for route in routes:
        route_distance = 0
        for i in range(len(route) - 1):
            route_distance += distanceMatrix[route[i]][route[i + 1]] 
        total_distance += route_distance
    return total_distance

# CVRPTWSolver

In [ ]:
class CVRPTWSolver:
    def __init__(self, distance_matrix, vehicle_number, vehicle_capacity, 
                 demands, time_windows, service_times, initial_solution=None,
                 time_window_penalty=1000, capacity_penalty=1000):
        self.distance_matrix = distance_matrix
        self.vehicle_number = vehicle_number
        self.vehicle_capacity = vehicle_capacity
        self.demands = demands
        self.time_windows = time_windows
        self.service_times = service_times
        self.time_window_penalty = time_window_penalty
        self.capacity_penalty = capacity_penalty
        self.num_customers = len(demands) - 1 
        
        if initial_solution is None:
            self.current_solution = self._create_initial_solution()
            print(f"create casual initial solution is {self.current_solution}")
        else:
            self.current_solution = copy.deepcopy(initial_solution)
        
        self.best_solution = copy.deepcopy(self.current_solution)
        self.best_cost = self._calculate_total_cost(self.best_solution)
        
    def _create_initial_solution(self):
        solution = []
        unassigned = list(range(1, self.num_customers + 1))
        
        for v in range(self.vehicle_number):
            if not unassigned:
                break
                
            route = [0]
            current_capacity = 0
            current_time = 0
            
            while unassigned:
                best_customer = None
                best_insertion_cost = float('inf')
                
                for customer in unassigned:
                    if current_capacity + self.demands[customer] > self.vehicle_capacity:
                        continue
                        
                    travel_time = self.distance_matrix[route[-1]][customer]
                    arrival_time = current_time + travel_time
                    
                    earliest, latest = self.time_windows[customer]
                    wait_time = max(0, earliest - arrival_time)
                    delay_time = max(0, arrival_time - latest)
                    
                    if delay_time > 0:
                        continue
                        
                    insertion_cost = travel_time + wait_time
                    
                    if insertion_cost < best_insertion_cost:
                        best_insertion_cost = insertion_cost
                        best_customer = customer
                
                if best_customer is None:
                    break
                    
                route.append(best_customer)
                unassigned.remove(best_customer)
                current_capacity += self.demands[best_customer]
                
                travel_time = self.distance_matrix[route[-2]][best_customer]
                arrival_time = current_time + travel_time
                earliest, latest = self.time_windows[best_customer]
                service_start = max(arrival_time, earliest)
                current_time = service_start + self.service_times[best_customer]
            
            route.append(0)
            solution.append(route)

        while unassigned:
            route = [0]
            customers_in_route = []
            current_capacity = 0
            
            for customer in sorted(unassigned, key=lambda x: self.time_windows[x][0]):
                if current_capacity + self.demands[customer] <= self.vehicle_capacity:
                    route.append(customer)
                    customers_in_route.append(customer)
                    current_capacity += self.demands[customer]
                    
            for customer in customers_in_route:
                unassigned.remove(customer)
                
            route.append(0)
            solution.append(route)
            
        return solution
    
    def _is_feasible_route(self, route):
        if not route or len(route) <= 2:
            return True, 0, 0
            
        total_demand = sum(self.demands[customer] for customer in route[1:-1])
        capacity_violation = max(0, total_demand - self.vehicle_capacity)
        
        current_time = 0
        time_window_violation = 0
        
        for i in range(1, len(route)):
            prev_node = route[i-1]
            curr_node = route[i]
            
            travel_time = self.distance_matrix[prev_node][curr_node]
            arrival_time = current_time + travel_time
            
            if curr_node != 0: 
                earliest, latest = self.time_windows[curr_node]
                
                service_start = max(arrival_time, earliest)
                delay = max(0, arrival_time - latest)
                time_window_violation += delay

                current_time = service_start + self.service_times[curr_node]
            else:
                current_time = arrival_time
        
        is_feasible = (capacity_violation == 0) and (time_window_violation == 0)
        return is_feasible, capacity_violation, time_window_violation
    
    def _calculate_route_cost(self, route):
        # if not route or len(route) <= 2:
        #     return 0
            
        if not isinstance(route, list) or len(route) <= 2:
            return 0  # Invalid or empty route

        # distance_cost = sum(self.distance_matrix[route[i-1]][route[i]] for i in range(1, len(route)))

        # Flatten the route if it's a list of lists
        if isinstance(route[0], list):
            route = [customer for subroute in route for customer in subroute]

        # Now calculate the distance cost assuming route is a flat list
        distance_cost = sum(self.distance_matrix[route[i-1]][route[i]] for i in range(1, len(route)))

        _, capacity_violation, time_window_violation = self._is_feasible_route(route)
        
        total_cost = distance_cost
        total_cost += self.capacity_penalty * capacity_violation
        total_cost += self.time_window_penalty * time_window_violation
        
        return total_cost
    
    def _calculate_total_cost(self, solution):
        return sum(self._calculate_route_cost(route) for route in solution)
    
    def _remove_customer(self, customer, solution):
        for i, route in enumerate(solution):
            if customer in route:
                route_index = i
                position = route.index(customer)
                route.pop(position)
                return solution, route_index
        return solution, -1
    
    def _best_insertion_position(self, customer, route):
        best_position = 0
        best_cost_change = float('inf')
        original_cost = self._calculate_route_cost(route)
        
        for i in range(1, len(route)):
            new_route = route.copy()
            new_route.insert(i, customer)
            new_cost = self._calculate_route_cost(new_route)
            cost_change = new_cost - original_cost
            
            if cost_change < best_cost_change:
                best_cost_change = cost_change
                best_position = i
                
        return best_position, best_cost_change
    
    def _relocate_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 2]
        if not non_empty_routes:
            return new_solution
            
        source_route_idx = random.choice(non_empty_routes)
        source_route = new_solution[source_route_idx]
        
        if len(source_route) <= 2:
            return new_solution
            
        customer_positions = list(range(1, len(source_route) - 1))
        if not customer_positions:
            return new_solution
            
        customer_pos = random.choice(customer_positions)
        customer = source_route[customer_pos]
        
        source_route.pop(customer_pos)
        
        best_route_idx = -1
        best_position = -1
        best_cost_change = float('inf')
        
        for route_idx, route in enumerate(new_solution):
            if len(route) < 2:
                continue
                
            position, cost_change = self._best_insertion_position(customer, route)
            
            if cost_change < best_cost_change:
                best_cost_change = cost_change
                best_position = position
                best_route_idx = route_idx
        
        if best_route_idx != -1:
            new_solution[best_route_idx].insert(best_position, customer)
        else:
            source_route.insert(customer_pos, customer)
            
        return new_solution
    
    def _exchange_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 2]
        if len(non_empty_routes) < 1:
            return new_solution
            
        route1_idx = random.choice(non_empty_routes)
        route1 = new_solution[route1_idx]
        
        if len(non_empty_routes) > 1 and random.random() < 0.5:
            remaining_routes = [i for i in non_empty_routes if i != route1_idx]
            route2_idx = random.choice(remaining_routes)
        else:
            route2_idx = route1_idx
            
        route2 = new_solution[route2_idx]
        
        customers1 = list(range(1, len(route1) - 1))
        if not customers1:
            return new_solution
            
        pos1 = random.choice(customers1)
        
        if route1_idx == route2_idx:
            remaining_positions = [i for i in range(1, len(route1) - 1) if i != pos1]
            if not remaining_positions:
                return new_solution
            pos2 = random.choice(remaining_positions)
        else:
            customers2 = list(range(1, len(route2) - 1))
            if not customers2:
                return new_solution
            pos2 = random.choice(customers2)
        
        customer1 = route1[pos1]
        customer2 = route2[pos2]
        
        route1[pos1] = customer2
        route2[pos2] = customer1
        
        return new_solution
    
    def _two_opt_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 3]
        if not non_empty_routes:
            return new_solution
            
        route_idx = random.choice(non_empty_routes)
        route = new_solution[route_idx]
        
        positions = list(range(1, len(route) - 1))
        if len(positions) < 2:
            return new_solution
            
        pos1 = random.choice(positions)
        remaining_pos = [p for p in positions if p != pos1]
        pos2 = random.choice(remaining_pos)
        
        if pos1 > pos2:
            pos1, pos2 = pos2, pos1
            
        new_solution[route_idx][pos1:pos2+1] = reversed(route[pos1:pos2+1])
        
        return new_solution
    
    def _or_opt_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 3]
        if not non_empty_routes:
            return new_solution
            
        route_idx = random.choice(non_empty_routes)
        route = new_solution[route_idx]
        
        seq_length = min(random.randint(1, 3), len(route) - 3)
        if seq_length <= 0:
            return new_solution
            
        max_start = len(route) - 1 - seq_length
        if max_start < 1:
            return new_solution
            
        start_pos = random.randint(1, max_start)
    
        sequence = route[start_pos:start_pos + seq_length]
        
        for _ in range(seq_length):
            route.pop(start_pos)
        
        possible_positions = [i for i in range(1, len(route)) if i != start_pos]
        if not possible_positions:
            for i, customer in enumerate(sequence):
                route.insert(start_pos + i, customer)
            return new_solution
            
        new_pos = random.choice(possible_positions)
        
        for i, customer in enumerate(sequence):
            route.insert(new_pos + i, customer)
            
        return new_solution
    
    def _route_fusion_operator(self, solution):

        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 2]
        if len(non_empty_routes) < 2:
            return new_solution
            
        route1_idx = random.choice(non_empty_routes)
        remaining_routes = [i for i in non_empty_routes if i != route1_idx]
        route2_idx = random.choice(remaining_routes)
        
        route1 = new_solution[route1_idx]
        route2 = new_solution[route2_idx]

        merged_route = [0] + route1[1:-1] + route2[1:-1] + [0]
        
        is_feasible, _, _ = self._is_feasible_route(merged_route)
        
        if is_feasible:
            new_solution[route1_idx] = merged_route
            new_solution.pop(route2_idx)
        
        return new_solution
    
    def _adaptive_large_neighborhood_search(self, current_solution, num_destroy=3, num_repair=2):
        solution = copy.deepcopy(current_solution)
        
        all_customers = []
        for route in solution:
            all_customers.extend([customer for customer in route[1:-1]])
            
        if not all_customers:
            return solution
        
        num_to_destroy = min(num_destroy, len(all_customers))
        to_remove = random.sample(all_customers, num_to_destroy)
        
        removed_customers = []
        for customer in to_remove:
            for i, route in enumerate(solution):
                if customer in route:
                    pos = route.index(customer)
                    route.pop(pos)
                    removed_customers.append(customer)
                    break
        
        for _ in range(num_repair):
            if not removed_customers:
                break
                
            customer = removed_customers.pop(random.randint(0, len(removed_customers) - 1))
            
            best_route_idx = -1
            best_position = -1
            best_cost_change = float('inf')
            
            for route_idx, route in enumerate(solution):
                if len(route) < 2:
                    continue
                    
                position, cost_change = self._best_insertion_position(customer, route)
                
                if cost_change < best_cost_change:
                    best_cost_change = cost_change
                    best_position = position
                    best_route_idx = route_idx
            
            if best_route_idx != -1:
                solution[best_route_idx].insert(best_position, customer)
            else:
                solution.append([0, customer, 0])
        
        for customer in removed_customers:
            solution.append([0, customer, 0])
            
        solution = [route for route in solution if len(route) > 2]
            
        return solution
    
    def simulated_annealing(self, initial_temp=1000, alpha=0.98, iterations_per_temp=100, min_temp=0.1, max_iterations=100000):
        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost
        
        temperature = initial_temp
        iteration = 0
        no_improvement = 0
        
        operators = [
            self._relocate_operator,
            self._exchange_operator,
            self._two_opt_operator,
            self._or_opt_operator,
            self._route_fusion_operator,
            self._adaptive_large_neighborhood_search
        ]
        
        weights = [1.0] * len(operators)
        
        success_count = [0] * len(operators)
        total_count = [0] * len(operators)
        
        while temperature > min_temp and iteration < max_iterations:
            for _ in range(iterations_per_temp):
                op_idx = random.choices(range(len(operators)), weights=weights, k=1)[0]
                operator = operators[op_idx]

                new_solution = operator(current_solution)
                new_cost = self._calculate_total_cost(new_solution)

                delta = new_cost - current_cost

                total_count[op_idx] += 1
                
                if delta < 0 or random.random() < np.exp(-delta / temperature):
                    current_solution = new_solution
                    current_cost = new_cost

                    success_count[op_idx] += 1

                    if current_cost < best_cost:
                        best_solution = copy.deepcopy(current_solution)
                        best_cost = current_cost
                        no_improvement = 0
                    else:
                        no_improvement += 1
                else:
                    no_improvement += 1
                
                iteration += 1

            for i in range(len(weights)):
                if total_count[i] > 0:
                    weights[i] = 0.5 * (success_count[i] / max(1, total_count[i])) + 0.5

            success_count = [0] * len(operators)
            total_count = [0] * len(operators)

            temperature *= alpha

            if no_improvement > 1000:
                current_solution = self._adaptive_large_neighborhood_search(best_solution, 
                                                                           num_destroy=min(10, len(best_solution)),
                                                                           num_repair=8)
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0
        
        return best_solution, best_cost
    
    def variable_neighborhood_search(self, max_iterations=1000, max_no_improvement=100):
        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost

        neighborhoods = [
            self._relocate_operator,
            self._exchange_operator,
            self._two_opt_operator,
            self._or_opt_operator,
            self._route_fusion_operator,
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=2, num_repair=2),
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=4, num_repair=3)
        ]
        
        iteration = 0
        no_improvement = 0
        k = 0
        
        while iteration < max_iterations and no_improvement < max_no_improvement:
            local_best_solution = copy.deepcopy(current_solution)
            local_best_cost = current_cost
            
            for _ in range(10):
                new_solution = neighborhoods[k](current_solution)
                new_cost = self._calculate_total_cost(new_solution)
                
                if new_cost < local_best_cost:
                    local_best_solution = new_solution
                    local_best_cost = new_cost

            if local_best_cost < current_cost:
                current_solution = local_best_solution
                current_cost = local_best_cost
                k = 0 

                if current_cost < best_cost:
                    best_solution = copy.deepcopy(current_solution)
                    best_cost = current_cost
                    no_improvement = 0
                else:
                    no_improvement += 1
            else:
                k = (k + 1) % len(neighborhoods)
                no_improvement += 1
            
            iteration += 1

            if k == 0 and no_improvement > 50:
                current_solution = self._adaptive_large_neighborhood_search(best_solution, 
                                                                           num_destroy=5, 
                                                                           num_repair=4)
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0
        
        return best_solution, best_cost
    
    def simulated_annealing_time_limited(
        self,
        time_limit=300,
        initial_temp=1000,
        alpha=0.98,
        iterations_per_temp=100,
        known_optimum=None
    ):
        import time
        start_time = time.time()

        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost

        temperature = initial_temp
        no_improvement = 0

        while True:
            if time.time() - start_time >= time_limit:
                break  

            for _ in range(iterations_per_temp):
                if time.time() - start_time >= time_limit:
                    break

                operators = [
                    self._relocate_operator,
                    self._exchange_operator,
                    self._two_opt_operator,
                    self._or_opt_operator,
                    self._route_fusion_operator,
                    self._adaptive_large_neighborhood_search
                ]
                operator = random.choice(operators)
                
                new_solution = operator(current_solution)
                new_cost = self._calculate_total_cost(new_solution)
                
                delta = new_cost - current_cost
                if delta < 0 or random.random() < np.exp(-delta / temperature):
                    current_solution = new_solution
                    current_cost = new_cost
                    if current_cost < best_cost:
                        best_solution = copy.deepcopy(current_solution)
                        best_cost = current_cost
                        no_improvement = 0
                    else:
                        no_improvement += 1
                else:
                    no_improvement += 1

                if known_optimum is not None and best_cost <= known_optimum:
                    return best_solution, best_cost

            temperature *= alpha
            if temperature < 0.1:
                temperature = 0.1
            
            if no_improvement > 1000:
                current_solution = self._adaptive_large_neighborhood_search(
                    best_solution,
                    num_destroy=10,
                    num_repair=8
                )
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0
        
        return best_solution, best_cost


    def variable_neighborhood_search_time_limited(
        self,
        time_limit=300,
        known_optimum=None
    ):
        import time
        start_time = time.time()

        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost

        neighborhoods = [
            self._relocate_operator,
            self._exchange_operator,
            self._two_opt_operator,
            self._or_opt_operator,
            self._route_fusion_operator,
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=2, num_repair=2),
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=4, num_repair=3)
        ]
        k = 0
        no_improvement = 0

        while True:
            if time.time() - start_time >= time_limit:
                break
            
            local_best_solution = copy.deepcopy(current_solution)
            local_best_cost = current_cost
            
            for _ in range(10):
                if time.time() - start_time >= time_limit:
                    break
                
                new_solution = neighborhoods[k](current_solution)
                new_cost = self._calculate_total_cost(new_solution)
                if new_cost < local_best_cost:
                    local_best_solution = new_solution
                    local_best_cost = new_cost
            
            if local_best_cost < current_cost:
                current_solution = local_best_solution
                current_cost = local_best_cost
                if current_cost < best_cost:
                    best_solution = copy.deepcopy(current_solution)
                    best_cost = current_cost
                    no_improvement = 0
                else:
                    no_improvement += 1
                k = 0
            else:
                k = (k + 1) % len(neighborhoods)
                no_improvement += 1

            if known_optimum is not None and best_cost <= known_optimum:
                return best_solution, best_cost
            
            if k == 0 and no_improvement > 50:
                current_solution = self._adaptive_large_neighborhood_search(
                    best_solution,
                    num_destroy=5,
                    num_repair=4
                )
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0

        return best_solution, best_cost


    
    def hybrid_metaheuristic(self, time_limit=300):
        start_time = time.time()

        sa_solution, sa_cost = self.simulated_annealing(
            initial_temp=100,
            alpha=0.98,
            iterations_per_temp=50,
            min_temp=1.0,
            max_iterations=1000
        )

        if sa_cost < self._calculate_total_cost(self.best_solution):
            self.best_solution = copy.deepcopy(sa_solution)
            self.best_cost = sa_cost

        if time.time() - start_time >= time_limit:
            return self.best_solution, self.best_cost

        self.current_solution = copy.deepcopy(self.best_solution)
        vns_solution, vns_cost = self.variable_neighborhood_search(
            max_iterations=2000,
            max_no_improvement=200
        )

        if vns_cost < self.best_cost:
            self.best_solution = copy.deepcopy(vns_solution)
            self.best_cost = vns_cost

        if time.time() - start_time >= time_limit:
            return self.best_solution, self.best_cost

        remaining_time = time_limit - (time.time() - start_time)
        iterations = min(500, int(remaining_time / 0.1))
        
        current_solution = copy.deepcopy(self.best_solution)
        current_cost = self.best_cost
        
        for _ in range(iterations):
            if time.time() - start_time >= time_limit:
                break

            destroy_size = random.randint(3, 8)
            repair_size = random.randint(2, destroy_size)
            
            new_solution = self._adaptive_large_neighborhood_search(
                current_solution,
                num_destroy=destroy_size,
                num_repair=repair_size
            )
            new_cost = self._calculate_total_cost(new_solution)
            
            if new_cost < current_cost:
                current_solution = new_solution
                current_cost = new_cost
                
                if new_cost < self.best_cost:
                    self.best_solution = copy.deepcopy(new_solution)
                    self.best_cost = new_cost
        
        return self.best_solution, self.best_cost

    # def solve(self, time_limit=300, method="hybrid"):
    #     start_time = time.time()
        
    #     if method == "sa":
    #         self.best_solution, self.best_cost = self.simulated_annealing(
    #             initial_temp=100,
    #             alpha=0.98,
    #             iterations_per_temp=100,
    #             min_temp=0.1,
    #             max_iterations=10000
    #         )
    #     elif method == "vns":
    #         self.best_solution, self.best_cost = self.variable_neighborhood_search(
    #             max_iterations=5000,
    #             max_no_improvement=500
    #         )
    #     else:  # hybrid method
    #         self.best_solution, self.best_cost = self.hybrid_metaheuristic(time_limit)
        
    #     end_time = time.time()
    #     solution_time = end_time - start_time
        
    #     solution_info = self._analyze_solution(self.best_solution)
    #     solution_info["total_cost"] = self.best_cost
    #     solution_info["solution_time"] = solution_time
    #     solution_info["method"] = method
        
    #     print(f"Solution completed, total cost: {self.best_cost:.2f}, time: {solution_time:.2f} seconds")
    #     return self.best_solution, self.best_cost, solution_info

    def solve(self, time_limit=300, method="vns", known_optimum=828.94):
        start_time = time.time()

        self.best_solution = copy.deepcopy(self.current_solution)
        self.best_cost = self._calculate_total_cost(self.best_solution)

        if method == "sa":
            best_solution, best_cost = self.simulated_annealing(
                initial_temp=1000,
                alpha=0.98,
                iterations_per_temp=100,
                min_temp=0.1,
                max_iterations=100000
            )
        elif method == "satl":
            best_solution, best_cost = self.simulated_annealing_time_limited(
                time_limit=time_limit,
                known_optimum=known_optimum
            )
        elif method == "vns":
            best_solution, best_cost = self.variable_neighborhood_search(
                max_iterations=1000,
                max_no_improvement=100
            )
        elif method == "vnstl":
            best_solution, best_cost = self.variable_neighborhood_search_time_limited(
                time_limit=time_limit
            )
        elif method == "hybrid":
            best_solution, best_cost = self.hybrid_metaheuristic(time_limit=time_limit)
        else:
            raise ValueError(f"unknow method：{method}")

        end_time = time.time()
        solution_time = end_time - start_time

        if best_cost < self.best_cost:
            self.best_solution = copy.deepcopy(best_solution)
            self.best_cost = best_cost

        # 输出或分析
        solution_info = self._analyze_solution(self.best_solution)
        solution_info["total_cost"] = self.best_cost
        solution_info["solution_time"] = solution_time
        solution_info["method"] = method

        print(f"Solution completed, total cost: {self.best_cost:.2f}, time: {solution_time:.2f} seconds")

        return self.best_solution, self.best_cost, solution_info


    def _analyze_solution(self, solution):
        info = {
            "num_routes": len(solution),
            "total_distance": 0,
            "total_time_window_violations": 0,
            "total_capacity_violations": 0,
            "route_info": []
        }
        
        for i, route in enumerate(solution):
            route_distance = sum(self.distance_matrix[route[j-1]][route[j]] for j in range(1, len(route)))
            _, capacity_violation, time_window_violation = self._is_feasible_route(route)
            
            info["total_distance"] += route_distance
            info["total_time_window_violations"] += time_window_violation
            info["total_capacity_violations"] += capacity_violation
            
            route_info = {
                "route_id": i,
                "route": route,
                "distance": route_distance,
                "customers": len(route) - 2, 
                "total_demand": sum(self.demands[customer] for customer in route[1:-1]),
                "capacity_violation": capacity_violation,
                "time_window_violation": time_window_violation,
                "is_feasible": (capacity_violation == 0 and time_window_violation == 0)
            }
            
            info["route_info"].append(route_info)
        
        if info["num_routes"] > 0:
            info["avg_customers_per_route"] = sum(r["customers"] for r in info["route_info"]) / info["num_routes"]
            info["avg_distance_per_route"] = info["total_distance"] / info["num_routes"]
        else:
            info["avg_customers_per_route"] = 0
            info["avg_distance_per_route"] = 0
            
        return info
    
    def visualize_solution(self, solution=None, coordinates=None, save_path=None):
        if solution is None:
            solution = self.best_solution
            
        if coordinates is None:
            print("No coordinates")
            return
            
        try:
            plt.figure(figsize=(12, 8))
            
            colors = list(mcolors.TABLEAU_COLORS.values())
            
            for node, (x, y) in coordinates.items():
                if node == 0:  
                    plt.scatter(x, y, c='red', s=100, marker='*', label='Depot')
                else:  
                    plt.scatter(x, y, c='blue', s=30)
                    plt.text(x, y, str(node), fontsize=8)
            
            for i, route in enumerate(solution):
                route_color = colors[i % len(colors)]
                route_x = [coordinates[node][0] for node in route]
                route_y = [coordinates[node][1] for node in route]
                
                plt.plot(route_x, route_y, c=route_color, linewidth=1.5, label=f'Route {i+1}')
            
            plt.title('CVRPTW Solution Visualization')
            plt.xlabel('X Coordinate')
            plt.ylabel('Y Coordinate')
            plt.legend(loc='best')
            plt.grid(True, linestyle='--', alpha=0.7)
            
            if save_path:
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                print(f"Image saved to {save_path}")
            else:
                plt.show()
                
        except ImportError:
            print("No matplotlib")

# produce train data

In [ ]:
def produce_train_data(file_path,time_limit=10, known_optimum=828.94):
    # Load the Solomon data
    def load_solomon_data(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Find vehicle information
        for i, line in enumerate(lines):
            if "VEHICLE" in line:
                vehicle_info_line = lines[i + 2]
                break

        # Extract vehicle number and capacity
        vehicle_number_line = vehicle_info_line.split()
        vehicle_number = int(vehicle_number_line[0])
        vehicle_capacity = int(vehicle_number_line[1])
        
        # Read customer information
        demands = []
        time_windows = []
        coords = []
        service_times = []

        for line in lines[i + 4:]:
            parts = line.split()
            if len(parts) == 7:
                customer_id = int(parts[0])
                x_coord = int(parts[1])
                y_coord = int(parts[2])
                demand = int(parts[3])
                ready_time = int(parts[4])
                due_time = int(parts[5])
                service_time = int(parts[6])

                demands.append(demand)
                time_windows.append((ready_time, due_time))
                coords.append((x_coord, y_coord))
                service_times.append(service_time)

        # Create a dictionary for coordinates
        coordinates_dict = {i: coord for i, coord in enumerate(coords)}
        
        return vehicle_number, vehicle_capacity, demands, time_windows, coords, service_times, coordinates_dict

    # Calculate distance matrix
    def calculate_distance_matrix(coords):
        n = len(coords)
        distance_matrix = [[0] * n for _ in range(n)]
        for i in range(n):
            for j in range(i + 1, n):
                dist = math.sqrt((coords[i][0] - coords[j][0]) ** 2 + (coords[i][1] - coords[j][1]) ** 2)
                distance_matrix[i][j] = dist
                distance_matrix[j][i] = dist
        return distance_matrix

    # Main execution
    if __name__ == "__main__":
        # Load data
        file_path = file_path  # Replace with your file path
        vehicle_number, vehicle_capacity, demands, time_windows, coords, service_times, coordinates_dict = load_solomon_data(file_path)
        
        # Generate distance matrix
        distance_matrix = calculate_distance_matrix(coords)
        
        # Initial solution that you provided
        #initial_solution = [[0, 20, 21, 24, 26, 29, 23, 27, 22, 30, 28, 25, 0], [0, 88, 85, 91, 87, 89, 86, 90, 0], [0, 47, 42, 48, 46, 52, 43, 40, 41, 49, 50, 44, 45, 51, 0], [0, 69, 66, 68, 63, 74, 72, 65, 64, 61, 67, 62, 0], [0, 8, 5, 3, 10, 2, 6, 7, 9, 4, 0], [0, 75, 76, 78, 1, 70, 81, 73, 71, 0], [0, 95, 98, 93, 94, 92, 99, 97, 100, 96, 0], [0, 82, 60, 58, 57, 54, 84, 59, 77, 55, 80, 56, 79, 53, 83, 0], [0, 32, 39, 37, 34, 36, 31, 33, 35, 38, 0], [0, 16, 14, 13, 18, 17, 19, 12, 15, 11, 0]]
        
        # Create and initialize the solver
        solver = CVRPTWSolver(
            distance_matrix=distance_matrix,
            vehicle_number=vehicle_number,
            vehicle_capacity=vehicle_capacity,
            demands=demands,
            time_windows=time_windows,
            service_times=service_times
        )
        
        # Solve the problem using the hybrid method (or choose another method)
        best_solution, best_cost, solution_info = solver.solve(time_limit=time_limit, method="satl",known_optimum=known_optimum)
        
        # Print the results
        print(f"Best solution cost: {best_cost}")
        print(f"Number of routes: {solution_info['num_routes']}")
        print(f"Total distance: {solution_info['total_distance']}")
        print(f"Time window violations: {solution_info['total_time_window_violations']}")
        print(f"Capacity violations: {solution_info['total_capacity_violations']}")
        
        # Print detailed route information
        total_route = []
        for i, route_info in enumerate(solution_info['route_info']):
            print(f"\nRoute {i+1}: {route_info['route']}")
            print(f"  Distance: {route_info['distance']:.2f}")
            print(f"  Customers: {route_info['customers']}")
            print(f"  Total demand: {route_info['total_demand']}")
            print(f"  Feasible: {route_info['is_feasible']}")

            if not all(x == 0 for x in route_info['route']):
                total_route.append(route_info['route'])
        
        print(f"Best route_info:{total_route}")

        # Visualize the solution
        #solver.visualize_solution(total_route, coordinates_dict)

    return total_route, best_cost

In [ ]:
file_folder = ["solomon_100"]
# file_folder = ["c0"]
time_list = [30, 60, 90, 120, 150]
base_path = "/home/wsl/CVRPTW/data/origine data"
save_path = "/home/wsl/CVRPTW/data/result_route_0402/c"

# Read the result_distance.xlsx file
result_distance_df = pd.read_excel("/home/wsl/CVRPTW/result_distance.xlsx")
# Convert to dictionary for easier lookup
result_distance_dict = dict(zip(result_distance_df['file_name'], result_distance_df['distance']))

all_conversations = []
for folder in file_folder:
    folder_path = os.path.join(base_path, folder)
    try:
        files = os.listdir(folder_path)  
        for file_name in files:
            file_link = os.path.join(folder_path, file_name)
            print(file_name)
            
            # Extract base name without extension for matching with result_distance.xlsx
            base_name = os.path.splitext(file_name)[0]
            
            # Get target distance from the excel file if available
            target_distance = None
            if base_name in result_distance_dict:
                target_distance = result_distance_dict[base_name]
                print(f"Target distance for {base_name}: {target_distance}")
            
            with open(file_link, 'r') as file:
                file_content = file.read() 
            
            i = 0
            distance_list = []
            pre_routes = []
            file_results = [] 
            for run_time in time_list:
                all_routes, total_distance = produce_train_data(file_link, run_time, target_distance)
                distance_list.append(total_distance)
                
                pre_routes = all_routes
                total_distance = round(total_distance, 2)
                i += 1
                file_results.append({
                    "run_time": run_time,
                    "all_routes": all_routes,
                    "total_distance": total_distance
                })
                
                
                # New break logic: if we reach the target distance from the excel file
                if run_time > 15 and target_distance is not None and total_distance == target_distance:  # Using exact equality
                    print(f"Reached target distance {target_distance} for {base_name}. Stopping further iterations.")
                    break
            
            result_file_path = os.path.join(save_path, f"{file_name}_results.txt")
            with open(result_file_path, 'w') as result_file:
                for result in file_results:
                    result_file.write(f"Run Time: {result['run_time']}\n")
                    result_file.write(f"All Routes: {json.dumps(result['all_routes'])}\n")
                    result_file.write(f"Total Distance: {result['total_distance']}\n\n")
    except FileNotFoundError:
        print(f"file folder {folder} not found")